### Musterlösung zu den Kontrollfragen des vorherigen Notebooks

* Wie ist in der Regel die Sensitivität einer Zählfunktion (`COUNT`)?
> Die Sensitivität einer Zählfunktion ist in aller Regel 1.

* Von welchen beiden Parametern ist die Laplace-Zufallsfunktion abhängig?
> Von der Sensitivität und dem Privacy-Budget: Lap(Sensitivität / Privacy-Budget)

<hr>

# Notebook 4: Der Gauss Mechanismus

***Hier geht es zum vorherigen Notebook dieser Serie: [Notebook 3: Der Laplace Mechanismus](./3_Laplace-Mechanismus.ipynb)***

## Lernziele

Folgende Lernziele sollten mit der Bearbeitung dieses Jupyter Notebooks erreicht werden können:
- Die Teilnehmenden sind in der Lage, die Definition der $(\varepsilon, \delta)$-Differential Privacy abzurufen.
- Die Teilnehmenden sind in der Lage, die Eigenschaften der Differential Privacy für eine einfache Funktion mittels des Gauss Mechanismus zu implementieren.
- Die Teilnehmenden sind in der Lage, die Vor- und Nachteile des Laplace und Gauss Mechanismus zu benennen.

## Die annähernde Differential Privacy

Die $\varepsilon$-Differential Privacy wird auch als **reine Differential Privacy** bezeichnet. In der Definition der **annähernden Differential Privacy** kommt ein $\delta$ dazu, welches eine kontrollierte Verletzung der reinen Differential Privacy erlaubt. Die $\varepsilon$-Differential Privacy kann auch als $(\varepsilon, \delta)$-Differential Privacy mit einem Wert von $\delta = 0$ definiert werden.

<div class="alert alert-success">
<b>Definition: $(\varepsilon, \delta)$-Differential Privacy</b>
    <br />
    <br /> 
Ein Mechanismus $M$ erfüllt dann die Eigenschaften von $(\varepsilon, \delta)$-Differential Privacy, wenn für alle Datensammlungen $D_1$ und $D_2$, welche sich in genau einem Datensatz unterscheiden, für alle möglichen Ausgaben $A$ gilt:

$$\mathbb{P}[M(D_1) = A] \leq e^{\varepsilon} * \mathbb{P}[M(D_2) = A] + \delta$$
</div>

Das $\delta$ kann auch als die Wahrscheinlichkeit, dass die $\varepsilon$-Differential Privacy verletzt wird, betrachtet werden. Somit kann es bei der annähernden Differential Privacy zu Ereignissen kommen, in welchen die $\varepsilon$-Differential Privacy nicht erfüllt wird.

Beim Gauss Mechanismus wird dieses $\delta$ benötigt, um extreme Ereignisse zu verhindern bzw. "Worst Case" Szenarien vorzubeugen. Die detaillierte Betrachtung der Bedeutung des $\delta$ würde aber den Umfang dieser Übung übersteigen.

## Definition des Gauss Mechanismus

Alternativ zum Laplace Mechanismus kann mit dem Gauss Mechanismus Rauschen nach der Gauss Verteilung (Normalverteilung) hinzugefügt werden. Gleich wie der Laplace Mechanismus wird auch der Gauss Mechanismus ausschliesslich für **numerische Abfragen** eingesetzt und liefert immer eine Zahl als Ausgabe zurück. Zur Veranschaulichung wird dieser Ablauf in <a href="#abb-1">Abbildung 1</a> grafisch gezeigt.

<br>
<br>
<center>
<img src="./src/Gauss-Mechanismus.png" width="50%" alt="Funktionsweise Laplace Mechanismus">
<br>
<br>
<a name="abb-1">Abbildung 1: Funktionsweise des Gauss Mechanismus</center></a>

Während der Laplace Mechanismus die Eigenschaften der reinen $\varepsilon$-Differential Privacy erfüllt, kann der Gauss Mechanismus nur die Eigenschaften der annähernden ($\varepsilon, \delta$)-Differential Privacy erfüllen.

<div class="alert alert-success">
<b>Definition: Gauss Mechanismus</b>
    <br />
    <br /> 
Zur Ausgabe der Funktion $f(D)$, welche eine Zahl zurück liefert, wird Rauschen nach der Gauss Verteilung $\mathcal{N}(0,\sigma^2)$ hinzugefügt. Dadurch erfüllt der Mechanismus $M(D)$ die Eigenschaften von ($\varepsilon, \delta$)-Differential Privacy.

$$M(D) = f(D) + \mathcal{N}(0, \sigma^2)$$
Wobei:
$$\sigma^2 = \frac{2*s^2*ln(1.25/\delta)}{\varepsilon^2}$$
</div>


Das $s$ entspricht der Sensitivität der Funktion $f(D)$. $\mathcal{N}(0, \sigma^2)$ bezeichnet das Gauss Rauschen, zentriert bei $0$ und mit der Varianz $\sigma^2$.

## Implementation des Gauss Mechanismus

Wir verwenden dasselbe Beispiel, wie beim Laplace Mechanismus in [Notebook 3](./3_Laplace-Mechanismus.ipynb). Nachfolgend werden wiederum die Daten geladen. Weiter wird die Zählfunktion implementiert, welche die Anzahl der Frauen in der Datensammlung zählt.

In [1]:
from array import *

datensammlung = [["Heinz", "Müller", 1958, "M"], 
                 ["Maria", "Meier", 1965, "F"], 
                 ["Jolanda", "Heine", 1968, "F"], 
                 ["Markus", "Inauen", 1978, "M"],
                 ["Sarah", "Hauser", 1994, "F"]]

def countFemales(array):
    count = 0
    for x in array:
        if x[3] == "F":
            count += 1
    return count

print(countFemales(datensammlung))

3


Der Skalenparameter der Gauss-Funktion (Normalverteilung) entspricht dem Sigma. Dieses Sigma wird unten in der Implementation zur besseren Übersichtlichkeit zuerst berechnet.

Es ist wichtig zu beachten, dass in der Definition des Gauss Mechanismus das $\sigma²$ angegeben ist Für die Funktion `np.random.normal` wird jedoch das $\sigma$ benötigt. Deshalb wird bei der Implementation (siehe nachfolgender Codeabschnitt) entsprechend die Wurzel `np.sqrt()` gezogen bzw. werden direkt das $s$ statt $s²$ und $\varepsilon$ statt $\varepsilon²$ eingesetzt.

In [2]:
import numpy as np

def dp_gauss_countFemales(array, sensitivity, epsilon, delta):
    sigma = (np.sqrt(2 * np.log(1.25 / delta)) * sensitivity) / epsilon
    return countFemales(array) + np.random.normal(loc=0, scale=sigma)

Wir führen den Mechanismus `dp_gauss_countFemales` wiederum 10 Mal aus und sehen, dass Rauschen hinzugefügt wird.

In [3]:
for i in range(10):
    print(dp_gauss_countFemales(datensammlung, 1, 0.1, 10e-5))

38.95999052807901
27.358901152729963
-39.287222427208924
3.382866122384327
-95.92951588138322
2.03248760613576
-93.72270707127657
25.242309287967338
79.65476753723696
68.82346364989279


<div class="alert alert-danger">
    <h2>Übung: Gauss Mechanismus</h2>
    <br />
Schreibe Code, welcher den Mechanismus "dp_gauss_countFemales" 1'000 Mal ausführt und den Durchschnitt der Ergebnisse berechnet. Probiere verschiedene Werte von Epsilon und Delta aus und beobachte die Ergebnisse.
<br />
<br />
<i>Hinweis: Eine Musterlösung ist im <a href="./7_Musterlösungen-der-Übungen.ipynb">Notebook 7: Musterlösungen der Übungen</a> zu finden.</i>
</div>

In [4]:
## Implementiere den Code hier

## Realitätsnahes Beispiel des Gauss Mechanismus

Das Beispiel aus dem vorherigen [Notebook 3](./3_Laplace-Mechanismus.ipynb) zur Studie "National Longitudinal Study of Youth, NLSY79" soll erneut aufgegriffen werden. Es wird wiederum die Funktion `lowEducationHighIncome` implementiert, welche die Anzahl Personen, mit einem Einkommen von über 33'761 USD und weniger als 16 Jahre in der Schule waren.

In [5]:
data = np.genfromtxt('./src/income.dat',
                     skip_header=0,
                     skip_footer=0,
                     names=True,
                     dtype=None,
                     delimiter=' ')

def lowEducationHighIncome(array):
    count = 0
    
    for x in array:
        if x[1] < 16 and x[2] > 33761:
            count += 1
    
    return count

print("Resultat ohne DP: " + str(lowEducationHighIncome(data)))

Resultat ohne DP: 882


Nun wird der Gauss Mechanismus implementiert.

In [6]:
def dp_gauss_lowEducationHighIncome(array, sensitivity, epsilon, delta):
    sigma = (np.sqrt(2 * np.log(1.25 / delta)) * sensitivity) / epsilon
    return lowEducationHighIncome(array) + np.random.normal(loc=0, scale=sigma)

print("Resultat mit DP: " + str(dp_gauss_lowEducationHighIncome(data, 1, 0.1, 10e-5)))

Resultat mit DP: 934.6333289474336


Auch beim Gauss Mechanismus weicht wie erwartet das "Resultat mit DP" vom "Resultat ohne DP" ab. 

Nachfolgend wird der Mechanismus 1000 mal ausgeführt und anschliessend wird die absolute mittlere Abweichung vom tatsächlichen Wert (882) gemessen und ausgegeben.

In [7]:
values = []

for i in range(1000):
    values.append(dp_gauss_lowEducationHighIncome(data, 1, 0.1, 10e-5))

print("Absolute mittlere Abweichung von 882: " + str(np.mean(np.absolute(values - np.asarray(882)))))

Absolute mittlere Abweichung von 882: 35.28585706273609


Das heisst, dass die Ergebnisse des Mechanismus im Schnitt um ~35 Personen vom tatsächlichen Wert abgewichen sind. Für ein besseres Verständnis betrachten wir die Zahlen in Prozent.

Total sind **2584 Personen** enthalten, dies entspricht **100%**  
Insgesamt treffen davon für **882 Personen** die angegebenen Kriterien zu, dies entspricht **34%**  
Die Ergebnisse des Mechanismus wichen im Schnitt um **~35 Personen** ab, was **1.4%** entspricht  

**Auswertung der Studie ohne Differential Privacy:**  
34.0% der Personen erzielen trotz einer geringeren Schulbildung ein hohes Einkommen.

**Auswertung der Studie mit Differential Privacy:**  
35.4% der Personen erzielen trotz einer geringeren Schulbildung ein hohes Einkommen.

Zur Erinnerung: Bei demselben Beispiel war beim Laplace Mechanismus die absolute mittlere Abweichung bei nur ~10 Personen, was 0.4% entspricht. Der Gauss Mechanismus fügt bei diesem Beispiel mehr Rauschen hinzu, als der Laplace Mechanismus.

Es stellt sich nun die Frage zu den Vor- und Nachteilen dieser beiden Mechanismen.

## Vergleich: Laplace und Gauss Mechanismus

Ein grosser Vorteil des Laplace Mechanismus ist, dass die Eigenschaften von $\varepsilon$-Differential Privacy in jedem Fall erfüllt werden. Im Gegensatz zum Gauss Mechanismus, bei welchem die Definition der Differential Privacy gelockert werden musste, indem ein $\delta$ hinzugefügt wurde. Möchte man somit die $\varepsilon$-Differential Privacy erreichen, kann der Gauss Mechanismus nicht verwendet werden.

Ein wesentlicher Vorteil des Gauss Mechanismus liegt in der Verteilung des Rauschens. Die Normalverteilung ist sehr verbreitet, intuitiv und hat viele gute statistische Eigenschaften.

Ein weiterer Unterschied liegt in der Menge des notwendigen Rauschens. Hat eine Person in der Datensammlung nur Einfluss auf wenige Statistiken, muss mit dem Laplace Mechanismus weniger Rauschen hinzugefügt werden. So wie dies auch im vorherigen Beispiel gezeigt werden konnte. Hat eine Person aber Einfluss auf viele Statistiken, so benötigt der Gauss Mechanismus weniger Rauschen. Desfontaines hat in <a href="https://desfontain.es/privacy/gaussian-noise.html">seinem Blog</a> eine Gegenüberstellung der beiden Mechanismen bezüglich der Menge des Rauschens gemacht.

<br>
<br>
<center>
<img src="./src/laplace-gaussian-comparison.png" alt="Vergleich Laplace und Gauss" width="60%">
<br>
<br>
<a name="abb-2">Abbildung 2: Vergleich des Rauschens beim Laplace und Gauss Mechanismus (Quelle: Desfontaines)</center></a>

In <a href="#abb-2">Abbildung 2</a> hat Desfontaines die Menge des Rauschens des Laplace und des Gauss Mechanismus bei einem $\varepsilon = 1.0$ und einem $\delta = 10^{-5}$ abgebildet. Es ist ersichtlich, dass bei dem gegebenen $\varepsilon$ und $\delta$ der Laplace Mechanismus weniger Rauschen benötigt, wenn eine Person in der Datensammlung Einfluss auf 1 bis ca. 7 Statistiken hat. Ab 7 Statistiken benötigte in diesem Beispiel der Gauss Mechanismus weniger Rauschen.

"Einfluss auf mehrere Statistiken" heisst in diesem Fall, dass eine Person Einfluss auf mehrere Werte haben kann, welche auf Basis derselben Datensammlung errechnet werden. 

Zur Verdeutlichung ein Beispiel: Die 50 meist geschauten Kinofilme sollen veröffentlicht werden. Dazu soll die Anzahl der Kinobesuche pro Film ausgegeben werden. Hat nun eine Person genau einen Kinofilm geschaut, so hat diese auch nur Einfluss auf genau einen Wert. Handelt es sich jedoch um eine Person, welche viele Kinofilme geschaut hat, so beeinflusst diese Person die Anzahl mehrerer Filme und dadurch auch mehrerer Statistiken. Bei vielen Statistiken wird das Rauschen dann beim Gauss Mechanismus geringer, als beim Laplace Mechanismus.

Schlussendlich hängt es also vom Anwendungsfall ab, ob nun der Laplace oder der Gauss Mechnismus implementiert wird. Dies muss entsprechend vor der Implementation gut überlegt werden.

## Kontrollfragen

* Ist der folgende Satz korrekt? "Der Schutz der Privacy ist besser, je kleiner das Privacy-Budget und je grösser das Delta."
* Wie wird die Gauss Verteilung auch genannt?
* Eine Person kommt potentiell in sehr vielen Statistiken (basierend auf demselben Mechanismus) vor. Wähle ich besser den Laplace oder den Gauss Mechanismus?

_Hinweis: Eine Musterlösung der Kontrollfragen ist im nächsten Notebook zu finden._

***Hier geht es zum nächsten Notebook dieser Serie: [Notebook 5: Der exponentielle Mechanismus](./5_Exponentieller-Mechanismus.ipynb)***